## Implementation of the functions

We try to implement the best subset , forward selection, and the lasso by Julia. Here are the results of implementation . 

We compare the output of $\beta$ coefficients and prediction of $y$ derived by our implementation and the function in R package `bestsubset` implemented by authors, using prostate data we have dealt with HW3 and HW4.

In [1]:
using CSV, Random
using DataFrames
data = CSV.read(download("https://web.stanford.edu/~hastie/ElemStatLearn/datasets/prostate.data"), DataFrame)

,Column1,lcavol,lweight,age,lbph,svi,lcp,gleason,pgg45
,Int64,Float64,Float64,Int64,Float64,Int64,Float64,Int64,Int64
1,1,-0.579818,2.76946,50,-1.38629,0,-1.38629,6,0
2,2,-0.994252,3.31963,58,-1.38629,0,-1.38629,6,0
3,3,-0.510826,2.69124,74,-1.38629,0,-1.38629,7,20
4,4,-1.20397,3.28279,58,-1.38629,0,-1.38629,6,0
5,5,0.751416,3.43237,62,-1.38629,0,-1.38629,6,0
6,6,-1.04982,3.22883,50,-1.38629,0,-1.38629,6,0
7,7,0.737164,3.47352,64,0.615186,0,-1.38629,6,0
8,8,0.693147,3.53951,58,1.53687,0,-1.38629,6,0
9,9,-0.776529,3.53951,47,-1.38629,0,-1.38629,6,0


In [2]:
include("bs.jl")
include("fs_new.jl")
include("lasso.jl")
include("lasso_glmnet.jl")

prediction_glmnet_df (generic function with 1 method)

In [3]:
X = Matrix(data[:, Not(["Column1","lpsa","train"])]) ;
y = Vector{Float64}(data[:, "lpsa"]) ;

using StatsBase
using Random, InvertedIndices
trainnum = Int(round(length(y) * 0.8))
Random.seed!(1)
trainobs = sample(1:length(y) ,trainnum, replace=false) ;
y_train = y[trainobs]
y_test = y[Not(trainobs)]
X_train = X[trainobs, :]
X_test = X[Not(trainobs), :] ;

## Best Subset

In [4]:
using RCall

R"""
library(bestsubset)
x <- $X_train
y <- $y_train
xtest <- $X_test
bs.obj = bs(x,y)
bs_beta = bs.obj$beta
bs_prediction  = predict(bs.obj,newx=xtest) 
"""
;

┌ Warning: RCall.jl: Loading required package: ggplot2
│ Loading required package: glmnet
│ Loading required package: Matrix
│ Loaded glmnet 4.1-3
│ Registered S3 method overwritten by 'bestsubset':
│   method     from   
│   predict.bs splines
└ @ RCall /Users/changtaeyeong/.julia/packages/RCall/iMDW2/src/io.jl:160


In [5]:
using Convex
using Gurobi
ENV["GRB_LICENSE_FILE"]="/Users/changtaeyeong/gurobi.lic" # set as YOUR path to license file
const GRB_ENV = Gurobi.Env()
const MOI = Convex.MOI

Academic license - for non-commercial use only


MathOptInterface

In [6]:
my_pred, my_betas = prediction_bs(X_train, y_train, X_test) ;

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only


In [7]:
@rget bs_beta   # Output from R `bestsubset` package

8×9 Matrix{Float64}:
 0.0  0.702004  0.626947  0.48429    0.498498   …   0.554311     0.558751
 0.0  0.0       0.698568  0.688224   0.758268       0.664061     0.65463
 0.0  0.0       0.0       0.0       -0.0125847     -0.0221346   -0.0220767
 0.0  0.0       0.0       0.0        0.0            0.0963484    0.0975977
 0.0  0.0       0.0       0.707141   0.706083       0.826482     0.824548
 0.0  0.0       0.0       0.0        0.0        …  -0.125668    -0.12749
 0.0  0.0       0.0       0.0        0.0            0.0         -0.0324888
 0.0  0.0       0.0       0.0        0.0            0.00418323   0.00479077

In [8]:
my_betas    # Output by our julia implementation

8×9 Matrix{Float64}:
 0.0  0.702007  0.626955  0.484286   0.498494   …   0.554315     0.558757
 0.0  0.0       0.698571  0.688214   0.758134       0.664139     0.654736
 0.0  0.0       0.0       0.0       -0.0125815     -0.0221352   -0.022074
 0.0  0.0       0.0       0.0        0.0            0.0963295    0.0975696
 0.0  0.0       0.0       0.707164   0.706196       0.826515     0.824589
 0.0  0.0       0.0       0.0        0.0        …  -0.125696    -0.127491
 0.0  0.0       0.0       0.0        0.0            0.0         -0.0325582
 0.0  0.0       0.0       0.0        0.0            0.00418272   0.00479027

In [9]:
@rget bs_prediction # Output from R `bestsubset` package

19×9 Matrix{Float64}:
 2.55277  0.850303  0.826545  0.994239  …  0.878219  0.842446  0.844371
 2.55277  2.06576   2.01956   1.93866      2.01185   1.96442   1.97542
 2.55277  1.70492   1.53734   1.53214      1.40917   1.35673   1.36506
 2.55277  1.25658   1.32657   1.40967      1.29695   1.3079    1.29882
 2.55277  1.67626   1.91749   1.91211      1.92675   1.8582    1.86508
 2.55277  2.79904   2.6591    2.42942   …  2.3672    2.46259   2.47746
 2.55277  1.81873   1.93437   1.90167      1.94856   1.88225   1.89139
 2.55277  2.56401   2.22077   2.04223      2.07552   2.00722   2.02451
 2.55277  2.5473    2.57787   2.39722      1.98966   1.83237   1.78231
 2.55277  2.5607    2.60764   2.424        2.27153   2.25453   2.23801
 2.55277  2.71369   3.13004   2.9096    …  3.1478    3.12206   3.13181
 2.55277  1.90828   1.99076   1.94021      1.96099   2.14343   2.16046
 2.55277  2.43767   2.91518   2.75036      2.9303    2.9574    2.94178
 2.55277  2.29328   2.48659   2.35555      2.55112   2

In [10]:
my_pred    # Output by our julia implementation

19×9 Matrix{Float64}:
 2.55277  0.850294  0.826525  0.994245  …  0.878247  0.842484  0.844405
 2.55277  2.06576   2.01955   1.93866      2.01187   1.96443   1.97545
 2.55277  1.70491   1.53733   1.53214      1.40921   1.35676   1.36511
 2.55277  1.25657   1.32656   1.40968      1.29696   1.30786   1.29875
 2.55277  1.67626   1.91749   1.91211      1.92677   1.85822   1.86512
 2.55277  2.79904   2.6591    2.42941   …  2.36712   2.46251   2.47739
 2.55277  1.81872   1.93436   1.90167      1.94858   1.88226   1.89142
 2.55277  2.56401   2.22076   2.04223      2.07555   2.00721   2.02452
 2.55277  2.5473    2.57787   2.39721      1.9896    1.83233   1.78227
 2.55277  2.5607    2.60764   2.424        2.27156   2.25459   2.23806
 2.55277  2.71369   3.13004   2.90958   …  3.14778   3.12211   3.13187
 2.55277  1.90827   1.99075   1.9402       1.96095   2.14337   2.16036
 2.55277  2.43767   2.91518   2.75035      2.93029   2.95745   2.94178
 2.55277  2.29328   2.48659   2.35554      2.5511    2

In [11]:
using BenchmarkTools

bm = @benchmark prediction_bs($X_train, $y_train, $X_test)

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic license - for non-commercial use only
Academic lice

BenchmarkTools.Trial: 4 samples with 1 evaluation.
 Range (min … max):  1.436 s …   1.585 s  ┊ GC (min … max): 0.62% … 0.59%
 Time  (median):     1.500 s              ┊ GC (median):    0.62%
 Time  (mean ± σ):   1.505 s ± 71.399 ms  ┊ GC (mean ± σ):  0.62% ± 0.02%

  █      █                                 █              █  
  █▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  1.44 s         Histogram: frequency by time        1.59 s <

 Memory estimate: 83.65 MiB, allocs estimate: 119271.

In [12]:
using RCall
R"""
library(microbenchmark)
Rbm <- microbenchmark(bs(x,y))
"""

RObject{VecSxp}
Unit: milliseconds
     expr      min       lq     mean   median       uq      max neval
 bs(x, y) 187.0863 196.4343 206.1053 202.0871 211.0265 297.8042   100


## Forward Selection

In [13]:
using RCall

R"""
library(bestsubset)
x <- $X_train
y <- $y_train
xtest <- $X_test
fs.obj = fs(x,y)
fs_beta = fs.obj$beta
fs_prediction  = predict(fs.obj,newx=xtest) 
"""
;

In [14]:
my_pred, my_betas = prediction_fs(X_train, y_train, X_test) ;

In [15]:
@rget fs_beta   # Output from R `bestsubset` package

8×9 Matrix{Float64}:
 0.0  0.702004  0.626947  0.48429    0.498498   …   0.554311     0.558751
 0.0  0.0       0.698568  0.688224   0.758268       0.664061     0.65463
 0.0  0.0       0.0       0.0       -0.0125847     -0.0221346   -0.0220767
 0.0  0.0       0.0       0.0        0.0            0.0963484    0.0975977
 0.0  0.0       0.0       0.707141   0.706083       0.826482     0.824548
 0.0  0.0       0.0       0.0        0.0        …  -0.125668    -0.12749
 0.0  0.0       0.0       0.0        0.0            0.0         -0.0324888
 0.0  0.0       0.0       0.0        0.0            0.00418323   0.00479077

In [16]:
my_betas    # Output by our julia implementation

8×9 Matrix{Float64}:
 0.0  0.702004  0.626947  0.48429    0.498498   …   0.554311     0.558751
 0.0  0.0       0.698568  0.688224   0.758268       0.664061     0.65463
 0.0  0.0       0.0       0.0       -0.0125847     -0.0221346   -0.0220767
 0.0  0.0       0.0       0.0        0.0            0.0963484    0.0975977
 0.0  0.0       0.0       0.707141   0.706083       0.826482     0.824548
 0.0  0.0       0.0       0.0        0.0        …  -0.125668    -0.12749
 0.0  0.0       0.0       0.0        0.0            0.0         -0.0324888
 0.0  0.0       0.0       0.0        0.0            0.00418323   0.00479077

In [17]:
@rget fs_prediction # Output from R `bestsubset` package

19×10 Matrix{Float64}:
 2.55277  0.850303  0.826545  0.994239  …  0.842446  0.844371  0.844371
 2.55277  2.06576   2.01956   1.93866      1.96442   1.97542   1.97542
 2.55277  1.70492   1.53734   1.53214      1.35673   1.36506   1.36506
 2.55277  1.25658   1.32657   1.40967      1.3079    1.29882   1.29882
 2.55277  1.67626   1.91749   1.91211      1.8582    1.86508   1.86508
 2.55277  2.79904   2.6591    2.42942   …  2.46259   2.47746   2.47746
 2.55277  1.81873   1.93437   1.90167      1.88225   1.89139   1.89139
 2.55277  2.56401   2.22077   2.04223      2.00722   2.02451   2.02451
 2.55277  2.5473    2.57787   2.39722      1.83237   1.78231   1.78231
 2.55277  2.5607    2.60764   2.424        2.25453   2.23801   2.23801
 2.55277  2.71369   3.13004   2.9096    …  3.12206   3.13181   3.13181
 2.55277  1.90828   1.99076   1.94021      2.14343   2.16046   2.16046
 2.55277  2.43767   2.91518   2.75036      2.9574    2.94178   2.94178
 2.55277  2.29328   2.48659   2.35555      2.69408   

In [18]:
my_pred    # Output by our julia implementation

19×9 Matrix{Float64}:
 2.55277  0.850303  0.826545  0.994239  …  0.878219  0.842446  0.844371
 2.55277  2.06576   2.01956   1.93866      2.01185   1.96442   1.97542
 2.55277  1.70492   1.53734   1.53214      1.40917   1.35673   1.36506
 2.55277  1.25658   1.32657   1.40967      1.29695   1.3079    1.29882
 2.55277  1.67626   1.91749   1.91211      1.92675   1.8582    1.86508
 2.55277  2.79904   2.6591    2.42942   …  2.3672    2.46259   2.47746
 2.55277  1.81873   1.93437   1.90167      1.94856   1.88225   1.89139
 2.55277  2.56401   2.22077   2.04223      2.07552   2.00722   2.02451
 2.55277  2.5473    2.57787   2.39722      1.98966   1.83237   1.78231
 2.55277  2.5607    2.60764   2.424        2.27153   2.25453   2.23801
 2.55277  2.71369   3.13004   2.9096    …  3.1478    3.12206   3.13181
 2.55277  1.90828   1.99076   1.94021      1.96099   2.14343   2.16046
 2.55277  2.43767   2.91518   2.75036      2.9303    2.9574    2.94178
 2.55277  2.29328   2.48659   2.35555      2.55112   2

In [19]:
using BenchmarkTools

bm = @benchmark prediction_fs($X_train, $y_train, $X_test)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  162.186 μs …  20.761 ms  ┊ GC (min … max):  0.00% … 97.74%
 Time  (median):     208.839 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   263.563 μs ± 526.108 μs  ┊ GC (mean ± σ):  11.25% ±  6.72%

  █▇▆▇▇▆▅▄▄▄▃▂▂▂▁▁▁                                             ▂
  ███████████████████▇▇▇▇▆▆▆▆▆▆▆▆▆▅▆▆▆▅▃▅▅▃▅▆▅▅▅▄▁▅▅▃▃▁▅▁▃▃▁▃▁▃ █
  162 μs        Histogram: log(frequency) by time        980 μs <

 Memory estimate: 386.52 KiB, allocs estimate: 871.

In [20]:
using RCall
R"""
library(microbenchmark)
Rbm <- microbenchmark(fs(x,y))
"""

RObject{VecSxp}
Unit: milliseconds
     expr     min       lq     mean   median      uq    max neval
 fs(x, y) 2.91032 2.991944 3.648699 3.212589 3.50117 11.146   100


## Lasso

In [21]:
include("lasso.jl")

prediction_lasso (generic function with 1 method)

In [22]:
using RCall

R"""
library(bestsubset)
x <- $X_train
y <- $y_train
xtest <- $X_test
lasso.obj = lasso(x,y)
lasso_beta = as.matrix(coef(lasso.obj))[-1, ] # store betas except intercepts
lasso_prediction  = predict(lasso.obj,newx=xtest)
"""
;

In [23]:
my_pred, my_betas , λs = prediction_lasso(X_train, y_train, X_test) ;

In [24]:
@rget lasso_beta   # Output from R `bestsubset` package

8×50 Matrix{Float64}:
 0.0  0.095222  0.177528  0.248177    0.30599    …   0.555562    0.555983
 0.0  0.0       0.0       0.0         0.0            0.654804    0.654808
 0.0  0.0       0.0       0.0         0.0           -0.0217357  -0.02178
 0.0  0.0       0.0       0.0         0.0            0.0964112   0.0965667
 0.0  0.0       0.0       0.00244138  0.0205741      0.819611    0.820316
 0.0  0.0       0.0       0.0         0.0        …  -0.122831   -0.123443
 0.0  0.0       0.0       0.0         0.0           -0.0254329  -0.0262866
 0.0  0.0       0.0       0.0         0.0            0.0045573   0.00458614

In [25]:
my_betas    # Output by our julia implementation

8×50 Matrix{Float64}:
 0.0  0.120293  0.219974  0.301141   0.344738  …   0.558326     0.558389
 0.0  0.0       0.0       0.0        0.0           0.654643     0.654647
 0.0  0.0       0.0       0.0        0.0          -0.0220339   -0.022041
 0.0  0.0       0.0       0.0        0.0           0.0974442    0.097469
 0.0  0.0       0.0       0.0070502  0.129264      0.823847     0.823951
 0.0  0.0       0.0       0.0        0.0       …  -0.126889    -0.126982
 0.0  0.0       0.0       0.0        0.0          -0.0316365   -0.0317618
 0.0  0.0       0.0       0.0        0.0           0.00476254   0.00476695

In [26]:
@rget lasso_prediction # Output from R `bestsubset` package

RObject{S4Sxp}
19 x 50 Matrix of class "dgeMatrix"
            s1       s2       s3       s4       s5       s6       s7       s8
 [1,] 2.552768 2.321840 2.122236 1.950276 1.805421 1.698960 1.604411 1.521913
 [2,] 2.552768 2.486709 2.429611 2.379973 2.335217 2.287392 2.241852 2.201170
 [3,] 2.552768 2.437763 2.338358 2.252405 2.177932 2.112699 2.040561 1.973261
 [4,] 2.552768 2.376949 2.224978 2.093905 1.982508 1.895647 1.827008 1.769720
 [5,] 2.552768 2.433876 2.331111 2.242274 2.165440 2.098826 2.063951 2.040958
 [6,] 2.552768 2.586173 2.615047 2.639205 2.654837 2.642388 2.618868 2.594518
 [7,] 2.552768 2.453201 2.367139 2.292639 2.227538 2.167796 2.127096 2.095364
 [8,] 2.552768 2.554294 2.555612 2.556118 2.552395 2.528607 2.476071 2.420600
 [9,] 2.552768 2.552026 2.551385 2.550209 2.545109 2.520515 2.502452 2.487841
[10,] 2.552768 2.553845 2.554775 2.554947 2.550951 2.527004 2.511045 2.498738
[11,] 2.552768 2.574596 2.593463 2.609032 2.617635 2.601068 2.626305 2.660555
[12,] 2.55276

In [27]:
my_pred    # Output by our julia implementation

19×50 Matrix{Float64}:
 2.55277  2.26104  2.0193   1.82065  1.68358  …  0.844949  0.844851  0.844778
 2.55277  2.46932  2.40016  2.34205  2.28047     1.97506   1.97512   1.97516
 2.55277  2.40748  2.28709  2.18726  2.10326     1.36527   1.36523   1.3652
 2.55277  2.33066  2.14661  1.99493  1.88309     1.29982   1.29965   1.29952
 2.55277  2.40257  2.27811  2.17496  2.08919     1.86512   1.86511   1.86511
 2.55277  2.59497  2.62994  2.6566   2.64056  …  2.47738   2.4774    2.47741
 2.55277  2.42699  2.32276  2.23608  2.15915     1.89129   1.8913    1.89131
 2.55277  2.5547   2.55629  2.55578  2.52515     2.02404   2.02412   2.02417
 2.55277  2.55183  2.55105  2.54861  2.51694     1.78603   1.7854    1.78493
 2.55277  2.55413  2.55525  2.55436  2.52352     2.23855   2.23846   2.23839
 2.55277  2.58034  2.60319  2.61999  2.59865  …  3.13071   3.1309    3.13104
 2.55277  2.44233  2.35082  2.27449  2.20313     2.15973   2.15986   2.15996
 2.55277  2.53305  2.5167   2.50159  2.4631      2.94

In [28]:
using BenchmarkTools

bm = @benchmark prediction_lasso($X_train, $y_train, $X_test)

BenchmarkTools.Trial: 188 samples with 1 evaluation.
 Range (min … max):  18.683 ms … 41.622 ms  ┊ GC (min … max):  0.00% … 33.45%
 Time  (median):     24.390 ms              ┊ GC (median):     0.00%
 Time  (mean ± σ):   26.696 ms ±  5.737 ms  ┊ GC (mean ± σ):  16.67% ± 15.54%

     ▃ █▃▇▄▅▄ ▄                   ▃ ▂   ▃                      
  ▃▃▇████████▆██▆▆▃▅▃▃▅▃▁▁▁▃▃▁▁▆▃▇█▇█▇▇▇█▇▃█▆▇▆▇▅▅▅▅▃▁▆▁▁▁▁▃▅ ▃
  18.7 ms         Histogram: frequency by time        38.9 ms <

 Memory estimate: 33.39 MiB, allocs estimate: 315411.

In [29]:
using RCall

R"""
library(microbenchmark)
Rbm <- microbenchmark(lasso(x,y))
"""

RObject{VecSxp}
Unit: milliseconds
        expr      min       lq     mean   median       uq      max neval
 lasso(x, y) 1.010305 1.016809 1.070416 1.023006 1.059173 1.643793   100


## Relaxed Lasso

In [30]:
using RCall

R"""
library(bestsubset)
x <- $X_train
y <- $y_train
xtest <- $X_test
relaxlasso.obj = lasso(x,y, nrelax = 3)
relaxlasso_beta = as.matrix(coef(relaxlasso.obj))[-1, ] # store betas except intercepts
relaxlasso_prediction  = predict(relaxlasso.obj,newx=xtest)
"""
;

In [31]:
my_pred, my_betas , λs , γs = prediction_lasso(X_train, y_train, X_test, nrelax = 3) ;

In [32]:
@rget relaxlasso_beta  # Output from R `bestsubset` package

8×150 Matrix{Float64}:
 0.0  0.0  0.0  0.095222  0.398613  0.702004  …   0.557367     0.558751
 0.0  0.0  0.0  0.0       0.0       0.0           0.654719     0.65463
 0.0  0.0  0.0  0.0       0.0       0.0          -0.0219284   -0.0220767
 0.0  0.0  0.0  0.0       0.0       0.0           0.0970822    0.0975977
 0.0  0.0  0.0  0.0       0.0       0.0           0.822432     0.824548
 0.0  0.0  0.0  0.0       0.0       0.0       …  -0.125466    -0.12749
 0.0  0.0  0.0  0.0       0.0       0.0          -0.0293877   -0.0324888
 0.0  0.0  0.0  0.0       0.0       0.0           0.00468846   0.00479077

In [33]:
my_betas

8×150 Matrix{Float64}:
 0.0  0.0  0.0  0.120293  0.411148  0.702004  …   0.55857      0.558751
 0.0  0.0  0.0  0.0       0.0       0.0           0.654638     0.65463
 0.0  0.0  0.0  0.0       0.0       0.0          -0.0220589   -0.0220767
 0.0  0.0  0.0  0.0       0.0       0.0           0.0975334    0.0975977
 0.0  0.0  0.0  0.0       0.0       0.0           0.824249     0.824548
 0.0  0.0  0.0  0.0       0.0       0.0       …  -0.127236    -0.12749
 0.0  0.0  0.0  0.0       0.0       0.0          -0.0321253   -0.0324888
 0.0  0.0  0.0  0.0       0.0       0.0           0.00477886   0.00479077

In [34]:
@rget relaxlasso_prediction # Output from R `bestsubset` package

19×150 Matrix{Float64}:
 2.55277  2.55277  2.55277  2.32184  1.58607  …  0.8472   0.845785  0.844371
 2.55277  2.55277  2.55277  2.48671  2.27624     1.9732   1.97431   1.97542
 2.55277  2.55277  2.55277  2.43776  2.07134     1.36599  1.36552   1.36506
 2.55277  2.55277  2.55277  2.37695  1.81676     1.30418  1.3015    1.29882
 2.55277  2.55277  2.55277  2.43388  2.05507     1.86516  1.86512   1.86508
 2.55277  2.55277  2.55277  2.58617  2.69261  …  2.47653  2.477     2.47746
 2.55277  2.55277  2.55277  2.4532   2.13596     1.89065  1.89102   1.89139
 2.55277  2.55277  2.55277  2.55429  2.55915     2.02177  2.02314   2.02451
 2.55277  2.55277  2.55277  2.55203  2.54966     1.80378  1.79305   1.78231
 2.55277  2.55277  2.55277  2.55384  2.55727     2.24129  2.23965   2.23801
 2.55277  2.55277  2.55277  2.5746   2.64414  …  3.12566  3.12874   3.13181
 2.55277  2.55277  2.55277  2.46535  2.18681     2.15567  2.15806   2.16046
 2.55277  2.55277  2.55277  2.53716  2.48741     2.94041  2.941

In [35]:
my_pred    # Output by our julia implementation

19×150 Matrix{Float64}:
 2.55277  2.55277  2.55277  2.26104  1.55567  …  0.844778  0.844574  0.844371
 2.55277  2.55277  2.55277  2.46932  2.26754     1.97516   1.97529   1.97542
 2.55277  2.55277  2.55277  2.40748  2.0562      1.3652    1.36513   1.36506
 2.55277  2.55277  2.55277  2.33066  1.79362     1.29952   1.29917   1.29882
 2.55277  2.55277  2.55277  2.40257  2.03942     1.86511   1.86509   1.86508
 2.55277  2.55277  2.55277  2.59497  2.697    …  2.47741   2.47744   2.47746
 2.55277  2.55277  2.55277  2.42699  2.12286     1.89131   1.89135   1.89139
 2.55277  2.55277  2.55277  2.5547   2.55935     2.02417   2.02434   2.02451
 2.55277  2.55277  2.55277  2.55183  2.54956     1.78493   1.78362   1.78231
 2.55277  2.55277  2.55277  2.55413  2.55742     2.23839   2.2382    2.23801
 2.55277  2.55277  2.55277  2.58034  2.64702  …  3.13104   3.13142   3.13181
 2.55277  2.55277  2.55277  2.44233  2.1753      2.15996   2.16021   2.16046
 2.55277  2.55277  2.55277  2.53305  2.48536     2.

In [36]:
using BenchmarkTools

bm = @benchmark prediction_lasso($X_train, $y_train, $X_test, nrelax=3)

BenchmarkTools.Trial: 163 samples with 1 evaluation.
 Range (min … max):  21.375 ms … 58.413 ms  ┊ GC (min … max):  0.00% … 22.30%
 Time  (median):     31.918 ms              ┊ GC (median):     0.00%
 Time  (mean ± σ):   30.675 ms ±  7.017 ms  ┊ GC (mean ± σ):  16.06% ± 14.15%

   ▄▅▂▂▅█               ▂▂▅▇▄▂▅                                
  ▆██████▅▇▅▆▅▆▁▅▁▃▃▁▃▇▇███████▅▅▆▆▃▃▃▃▁▃▁▅▁▃▃▁▃▃▁▁▁▁▁▃▁▅▁▁▃▃ ▃
  21.4 ms         Histogram: frequency by time        49.6 ms <

 Memory estimate: 37.53 MiB, allocs estimate: 319841.

In [37]:
using RCall

R"""
library(microbenchmark)
Rbm <- microbenchmark(lasso(x,y, nrelax = 3))
"""

RObject{VecSxp}
Unit: milliseconds
                    expr      min       lq    mean   median       uq     max
 lasso(x, y, nrelax = 3) 1.013545 1.145029 1.53325 1.351107 1.701205 3.57826
 neval
   100


## Lasso & Relaxed Lasso using glmnet wrapper function

In [38]:
using RCall

R"""
library(bestsubset)
x <- $X_train
y <- $y_train
xtest <- $X_test
lasso.obj = lasso(x,y)
lasso_beta = as.matrix(coef(lasso.obj))[-1, ] # store betas except intercepts
lasso_prediction  = predict(lasso.obj,newx=xtest)
"""
;

In [39]:
my_pred, my_betas , λs = prediction_glmnet(X_train, y_train, X_test, nlambda = 50) ;

In [40]:
@rget lasso_beta   # Output from R `bestsubset` package

8×50 Matrix{Float64}:
 0.0  0.095222  0.177528  0.248177    0.30599    …   0.555562    0.555983
 0.0  0.0       0.0       0.0         0.0            0.654804    0.654808
 0.0  0.0       0.0       0.0         0.0           -0.0217357  -0.02178
 0.0  0.0       0.0       0.0         0.0            0.0964112   0.0965667
 0.0  0.0       0.0       0.00244138  0.0205741      0.819611    0.820316
 0.0  0.0       0.0       0.0         0.0        …  -0.122831   -0.123443
 0.0  0.0       0.0       0.0         0.0           -0.0254329  -0.0262866
 0.0  0.0       0.0       0.0         0.0            0.0045573   0.00458614

In [41]:
my_betas    # Output by our julia implementation

8×39 CompressedPredictorMatrix:
 0.0  0.120293  0.219974  0.301155    0.344767  …   0.555429     0.555983
 0.0  0.0       0.0       0.0         0.0           0.654803     0.654808
 0.0  0.0       0.0       0.0         0.0          -0.0217216   -0.02178
 0.0  0.0       0.0       0.0         0.0           0.0963619    0.0965667
 0.0  0.0       0.0       0.00702739  0.129219      0.819388     0.820316
 0.0  0.0       0.0       0.0         0.0       …  -0.122637    -0.123443
 0.0  0.0       0.0       0.0         0.0          -0.0251617   -0.0262866
 0.0  0.0       0.0       0.0         0.0           0.00454813   0.00458614

In [42]:
@rget lasso_prediction  # Output from R `bestsubset` package

RObject{S4Sxp}
19 x 50 Matrix of class "dgeMatrix"
            s1       s2       s3       s4       s5       s6       s7       s8
 [1,] 2.552768 2.321840 2.122236 1.950276 1.805421 1.698960 1.604411 1.521913
 [2,] 2.552768 2.486709 2.429611 2.379973 2.335217 2.287392 2.241852 2.201170
 [3,] 2.552768 2.437763 2.338358 2.252405 2.177932 2.112699 2.040561 1.973261
 [4,] 2.552768 2.376949 2.224978 2.093905 1.982508 1.895647 1.827008 1.769720
 [5,] 2.552768 2.433876 2.331111 2.242274 2.165440 2.098826 2.063951 2.040958
 [6,] 2.552768 2.586173 2.615047 2.639205 2.654837 2.642388 2.618868 2.594518
 [7,] 2.552768 2.453201 2.367139 2.292639 2.227538 2.167796 2.127096 2.095364
 [8,] 2.552768 2.554294 2.555612 2.556118 2.552395 2.528607 2.476071 2.420600
 [9,] 2.552768 2.552026 2.551385 2.550209 2.545109 2.520515 2.502452 2.487841
[10,] 2.552768 2.553845 2.554775 2.554947 2.550951 2.527004 2.511045 2.498738
[11,] 2.552768 2.574596 2.593463 2.609032 2.617635 2.601068 2.626305 2.660555
[12,] 2.55276

In [43]:
my_pred  # Output by our julia implementation

19×39 Matrix{Float64}:
 2.55277  2.26104  2.0193   1.82062  1.68352  …  0.848702  0.847819  0.8472
 2.55277  2.46932  2.40016  2.34204  2.28046     1.97244   1.9728    1.9732
 2.55277  2.40748  2.28709  2.18724  2.10324     1.36661   1.36622   1.36599
 2.55277  2.33066  2.14661  1.99491  1.88305     1.30655   1.30527   1.30418
 2.55277  2.40257  2.27811  2.17495  2.08917     1.86534   1.8652    1.86516
 2.55277  2.59497  2.62994  2.65661  2.64058  …  2.47644   2.47645   2.47653
 2.55277  2.42699  2.32276  2.23607  2.15913     1.89049   1.89053   1.89065
 2.55277  2.5547   2.55629  2.55579  2.52516     2.02077   2.02128   2.02177
 2.55277  2.55183  2.55105  2.54862  2.51695     1.81218   1.80791   1.80378
 2.55277  2.55413  2.55525  2.55437  2.52353     2.24245   2.24188   2.24129
 2.55277  2.58034  2.60319  2.62     2.59867  …  3.1231    3.12445   3.12566
 2.55277  2.44233  2.35082  2.27448  2.20311     2.15412   2.15483   2.15567
 2.55277  2.53305  2.5167   2.50159  2.46311     2.9396

In [44]:
using BenchmarkTools

bm = @benchmark prediction_glmnet($X_train, $y_train, $X_test, nlambda=50)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  47.982 μs …   8.696 ms  ┊ GC (min … max): 0.00% … 99.04%
 Time  (median):     52.410 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   59.079 μs ± 201.669 μs  ┊ GC (mean ± σ):  8.32% ±  2.43%

  ▃▅▆█▇██▆▅▄▂▁                                                 ▂
  ███████████████▆█▇▇█▇▇▇▇▆▆▆▆▅▆▅▆▅▆▆▅▆▇▇▇▇▇▅▇▆▅▆▆▅▄▅▆▄▅▅▅▄▅▅▅ █
  48 μs         Histogram: log(frequency) by time      96.4 μs <

 Memory estimate: 51.25 KiB, allocs estimate: 68.

In [45]:
using RCall

R"""
library(microbenchmark)
Rbm <- microbenchmark(lasso(x,y))
"""

RObject{VecSxp}
Unit: milliseconds
        expr     min       lq     mean   median       uq      max neval
 lasso(x, y) 1.08866 1.235629 1.602025 1.405532 1.605285 6.464913   100


In [46]:
using RCall

R"""
library(bestsubset)
x <- $X_train
y <- $y_train
xtest <- $X_test
relaxlasso.obj = lasso(x,y, nrelax = 3)
relaxlasso_beta = as.matrix(coef(relaxlasso.obj))[-1, ] # store betas except intercepts
relaxlasso_prediction  = predict(relaxlasso.obj,newx=xtest)
"""
;

In [47]:
my_pred, my_betas , λs = prediction_glmnet(X_train, y_train, X_test, nlambda = 50, nrelax = 3) ;

In [48]:
@rget relaxlasso_beta  # Output from R `bestsubset` package

8×150 Matrix{Float64}:
 0.0  0.0  0.0  0.095222  0.398613  0.702004  …   0.557367     0.558751
 0.0  0.0  0.0  0.0       0.0       0.0           0.654719     0.65463
 0.0  0.0  0.0  0.0       0.0       0.0          -0.0219284   -0.0220767
 0.0  0.0  0.0  0.0       0.0       0.0           0.0970822    0.0975977
 0.0  0.0  0.0  0.0       0.0       0.0           0.822432     0.824548
 0.0  0.0  0.0  0.0       0.0       0.0       …  -0.125466    -0.12749
 0.0  0.0  0.0  0.0       0.0       0.0          -0.0293877   -0.0324888
 0.0  0.0  0.0  0.0       0.0       0.0           0.00468846   0.00479077

In [49]:
my_betas

8×117 Matrix{Float64}:
 0.0  0.0  0.0  0.120293  0.411148  0.702004  …   0.557367     0.558751
 0.0  0.0  0.0  0.0       0.0       0.0           0.654719     0.65463
 0.0  0.0  0.0  0.0       0.0       0.0          -0.0219284   -0.0220767
 0.0  0.0  0.0  0.0       0.0       0.0           0.0970822    0.0975977
 0.0  0.0  0.0  0.0       0.0       0.0           0.822432     0.824548
 0.0  0.0  0.0  0.0       0.0       0.0       …  -0.125466    -0.12749
 0.0  0.0  0.0  0.0       0.0       0.0          -0.0293877   -0.0324888
 0.0  0.0  0.0  0.0       0.0       0.0           0.00468846   0.00479077

In [50]:
@rget relaxlasso_prediction # Output from R `bestsubset` package

19×150 Matrix{Float64}:
 2.55277  2.55277  2.55277  2.32184  1.58607  …  0.8472   0.845785  0.844371
 2.55277  2.55277  2.55277  2.48671  2.27624     1.9732   1.97431   1.97542
 2.55277  2.55277  2.55277  2.43776  2.07134     1.36599  1.36552   1.36506
 2.55277  2.55277  2.55277  2.37695  1.81676     1.30418  1.3015    1.29882
 2.55277  2.55277  2.55277  2.43388  2.05507     1.86516  1.86512   1.86508
 2.55277  2.55277  2.55277  2.58617  2.69261  …  2.47653  2.477     2.47746
 2.55277  2.55277  2.55277  2.4532   2.13596     1.89065  1.89102   1.89139
 2.55277  2.55277  2.55277  2.55429  2.55915     2.02177  2.02314   2.02451
 2.55277  2.55277  2.55277  2.55203  2.54966     1.80378  1.79305   1.78231
 2.55277  2.55277  2.55277  2.55384  2.55727     2.24129  2.23965   2.23801
 2.55277  2.55277  2.55277  2.5746   2.64414  …  3.12566  3.12874   3.13181
 2.55277  2.55277  2.55277  2.46535  2.18681     2.15567  2.15806   2.16046
 2.55277  2.55277  2.55277  2.53716  2.48741     2.94041  2.941

In [51]:
my_pred    # Output by our julia implementation

19×117 Matrix{Float64}:
 2.55277  2.55277  2.55277  2.26104  1.55567  …  0.8472   0.845785  0.844371
 2.55277  2.55277  2.55277  2.46932  2.26754     1.9732   1.97431   1.97542
 2.55277  2.55277  2.55277  2.40748  2.0562      1.36599  1.36552   1.36506
 2.55277  2.55277  2.55277  2.33066  1.79362     1.30418  1.3015    1.29882
 2.55277  2.55277  2.55277  2.40257  2.03942     1.86516  1.86512   1.86508
 2.55277  2.55277  2.55277  2.59497  2.697    …  2.47653  2.477     2.47746
 2.55277  2.55277  2.55277  2.42699  2.12286     1.89065  1.89102   1.89139
 2.55277  2.55277  2.55277  2.5547   2.55935     2.02177  2.02314   2.02451
 2.55277  2.55277  2.55277  2.55183  2.54956     1.80378  1.79305   1.78231
 2.55277  2.55277  2.55277  2.55413  2.55742     2.24129  2.23965   2.23801
 2.55277  2.55277  2.55277  2.58034  2.64702  …  3.12566  3.12874   3.13181
 2.55277  2.55277  2.55277  2.44233  2.1753      2.15567  2.15806   2.16046
 2.55277  2.55277  2.55277  2.53305  2.48536     2.94041  2.941